In [25]:
FILE_NAME = '\youTubeVideoData_680473'

In [26]:
import os
import sys

absolutepath = os.path.dirname(os.path.abspath("__file__"))
print(absolutepath)

fileDirectory = os.path.dirname(absolutepath)
print(fileDirectory)#Path of parent directory
parentDirectory = os.path.dirname(fileDirectory)
print(parentDirectory)
#Navigate to Strings directory
newPath = os.path.join(parentDirectory, 'data', 'raw')   
print(newPath)

C:\Users\vgarg\Documents\GitHub\Recommendation-System-for-YT\notebook\eda
C:\Users\vgarg\Documents\GitHub\Recommendation-System-for-YT\notebook
C:\Users\vgarg\Documents\GitHub\Recommendation-System-for-YT
C:\Users\vgarg\Documents\GitHub\Recommendation-System-for-YT\data\raw


In [27]:
import zipfile

In [28]:
# unzipping the datafile from data raw folder to 
with zipfile.ZipFile(newPath+FILE_NAME+'.zip',"r") as zip_ref:
    zip_ref.extractall(absolutepath)

In [29]:
# os.path.dirname(os.path.abspath("__file__"))

In [30]:
import pandas as pd

In [31]:
df = pd.read_csv(absolutepath+FILE_NAME+'.csv')

C:\Users\vgarg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [32]:
df

,Unnamed: 0,ContentId,title,viewCount,likeCount,dislikeCount,commentCount,favoriteCount,publishedAt,description,topicCategories,duration,tags
0,0,uzl8Fooi_LE,"Jodaa (Official Video) Jatinder Shah, Afsana K...",10814444,298544,11821,36687,0,2021-10-11T05:30:09Z,Universal Music proudly brings to you legendar...,"['https://en.wikipedia.org/wiki/Music', 'https...",PT4M52S,"['vyrl originals', 'vyrl original', 'jodaa', '..."
1,1,w8yoR0IMs08,"Jodaa (Official Teaser) Jatinder Shah, Afsana ...",1432519,48398,972,3475,0,2021-10-09T05:30:03Z,Universal Music brings to you the teaser of th...,['https://en.wikipedia.org/wiki/Entertainment'],PT29S,"['vyrl originals', 'vyrl original', 'jodaa', '..."
2,2,JngV9Ix5hnk,"Majnu (Official Video) Sukriti, Prakriti, Mell...",20686975,297683,15042,22395,0,2021-09-30T05:30:13Z,After all the love you have given their hit si...,"['https://en.wikipedia.org/wiki/Music', 'https...",PT3M30S,"['vyrl originals', 'vyrl original', 'sukriti',..."
3,3,8AOZLArpvOA,"Majnu (Official Teaser) Sukriti, Prakriti, Mel...",531332,21739,654,1592,0,2021-09-28T05:32:03Z,This one's for all the Majnu's in the house! 😉...,"['https://en.wikipedia.org/wiki/Music', 'https...",PT29S,"['sukriti', 'sukriti kakar', 'prakriti', 'prak..."
4,4,C7mF13gjjd0,Vishal Mishra - Aaj Bhi (WORMONO LoFi Remake) ...,386427,17893,274,407,0,2021-09-27T05:30:09Z,VYRL Originals presents WORMONO LoFi Remake of...,"['https://en.wikipedia.org/wiki/Music', 'https...",PT3M37S,"['vyrl originals', 'Aaj Bhi', 'vishal mishra',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
680468,149522,-FqSE-MLldA,"Anderson and Broad, hardest bowlers to face in...",4655,125,4,27,0,2020-07-26T04:19:14Z,"On #CricbuzzChatter, Adam Collins and Michael ...","['https://en.wikipedia.org/wiki/Cricket', 'htt...",PT8M,[]
680469,150399,-oH8flXJudw,Within twenty minutes I realised we picked a g...,15018,439,12,58,0,2020-07-11T14:30:00Z,Former #SRH coach Tom Moody shares an interest...,"['https://en.wikipedia.org/wiki/Cricket', 'htt...",PT3M52S,[]
680470,150416,-tV9enWCjxg,"England vs West Indies, 1st Test, Day 1: Match...",8493,261,14,11,0,2020-07-08T20:03:11Z,It rained on cricket's resumption parade as th...,"['https://en.wikipedia.org/wiki/Cricket', 'htt...",PT3M21S,"['ENG v WI', 'Cricket', 'ben Stokes', 'England..."
680471,161424,-OZPHH8UMwc,Revealed: How Babar Azam perfected his signatu...,25228,1510,23,110,0,2020-06-13T11:30:00Z,An innocent childhood routine is the reason be...,"['https://en.wikipedia.org/wiki/Cricket', 'htt...",PT3M43S,"['Cricket', 'IPL', 'T20', 'Pakistan', 'Babar A..."
